In [17]:
import numpy as np
import pandas as pd


path = './data/'
player_ext = 'Players_2010.csv'
event_ext = 'Events_2010.csv'
teams_ext = 'Teams.csv'

events = pd.read_csv(path+event_ext)
players = pd.read_csv(path+player_ext)
teams = pd.read_csv(path+teams_ext)



In [15]:
list(events)

['EventID',
 'Season',
 'DayNum',
 'WTeamID',
 'LTeamID',
 'WPoints',
 'LPoints',
 'ElapsedSeconds',
 'EventTeamID',
 'EventPlayerID',
 'EventType']

In [18]:
# make sure score keeping is relatively consistent
# events.groupby(['EventType'])['EventID'].count()

# 2018 numbers
#made2_dunk     18975
# made2_jump     64606
# made2_lay     110309
# made2_tip       4198

# 2010 numbers
# made2_dunk     16065
# made2_jump     69599
# made2_lay      97364
# made2_tip       4416

# seems okay, scoring seems to have gone up over time, or there are more games

EventType
assist        135116
block          36076
foul_pers     196038
foul_tech       1486
made1_free    151741
made2_dunk     16065
made2_jump     69599
made2_lay      97364
made2_tip       4416
made3_jump     65164
miss1_free     59367
miss2_dunk      1829
miss2_jump    128469
miss2_lay      72835
miss2_tip       2447
miss3_jump    125362
reb_dead       32061
reb_def       247105
reb_off       119795
steal          69508
sub_in        325212
sub_out       326376
timeout        36260
timeout_tv     38518
turnover      144421
Name: EventID, dtype: int64

In [19]:
# only use events we care about
# print(list(events.EventType.unique()))

dont_care = ['foul_pers','foul_tech','timeout_tv', 'reb_dead', 'timeout','sub_in', 'sub_out']
# i don't think i'm going to tally missed shots
missed = ['miss2_jump','miss1_free','miss3_jump', 'miss2_lay',  'miss2_dunk', 'miss2_tip']

event_types = ['assist', 'made3_jump', 'turnover',  'reb_off', 'made2_dunk','made2_jump', 'steal', 'made1_free', 'block','reb_def','made2_lay', 'made2_tip']



2502630
1156370


In [ ]:
def filter_events(df):
    df = df.loc[df.EventType.isin(event_types)]
    return df



In [22]:
# categories I'm interested in year over yeat
# aggregate event types
rebounds = ['reb_off','reb_def']
# close range 2pts
cr_2pt = ['made2_lay', 'made2_tip', 'made2_dunk']
# mid range 2pts
mr_2pt = ['made2_jump']
# 3 points
lr = ['made3_jump']
# assists, blocks, steals, fts
# 8 total categories

In [23]:
def aggregate_events(df):
    
    df.loc[df.EventType.isin(rebounds), 'EventType'] = 'rebound'
    df.loc[df.EventType.isin(cr_2pt), 'EventType'] = 'cr_2pt'
    
    return df



In [27]:
# can group events by player id

def player_level_events(df):
    df = df.groupby(['EventPlayerID','EventType'])['EventID'].count().unstack(-1).reset_index().fillna(0)
    return df


EventType,EventPlayerID,assist,block,cr_2pt,made1_free,made2_jump,made3_jump,rebound,steal,turnover
0,600001,7.0,0.0,10.0,4.0,2.0,2.0,17.0,7.0,6.0
1,600002,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,600003,11.0,9.0,14.0,25.0,6.0,1.0,54.0,4.0,22.0
3,600004,6.0,6.0,15.0,15.0,8.0,10.0,25.0,6.0,11.0
4,600005,1.0,0.0,0.0,1.0,1.0,0.0,4.0,0.0,0.0
5,600006,5.0,0.0,1.0,6.0,3.0,22.0,21.0,2.0,10.0
6,600007,63.0,3.0,23.0,20.0,10.0,14.0,41.0,22.0,35.0
7,600008,18.0,1.0,20.0,33.0,18.0,44.0,70.0,13.0,16.0
8,600010,14.0,0.0,2.0,0.0,3.0,1.0,15.0,2.0,9.0
9,600011,48.0,7.0,32.0,25.0,28.0,19.0,84.0,19.0,45.0


In [12]:
# first generate list of roster turnover

# years = list(range(2010, 2020))
# years = years.astype(str)

# for y in years:
#     player_ext = "Players_{}.csv".format(y)
#     players = pd.read_csv(path+player_ext)
#     players = players.rename('')
    



[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]


['assist', 'made3_jump', 'turnover', 'miss2_jump', 'reb_off', 'made2_dunk', 'foul_pers', 'made2_jump', 'steal', 'made1_free', 'sub_in', 'sub_out', 'block', 'reb_def', 'miss1_free', 'made2_lay', 'miss3_jump', 'miss2_lay', 'timeout_tv', 'reb_dead', 'timeout', 'miss2_dunk', 'made2_tip', 'miss2_tip', 'foul_tech']


In [13]:
players.head()

,PlayerID,Season,TeamID,PlayerName
0,648095,2018,1101,COBB_CHASE
1,648096,2018,1101,FARQHAR_HAYDEN
2,648097,2018,1101,FARQUHAR_HAYDEN
3,648098,2018,1101,FOSTER_TEVIN
4,648099,2018,1101,FRANKLIN_JAYLEN


In [8]:
# roster = events.groupby(['EventTeamID','EventPlayerID','EventType'])['EventID'].count().reset_index()
# roster.head(40)

,EventTeamID,EventPlayerID,EventType,EventID
0,1101,648095,miss3_jump,1
1,1101,648095,sub_in,2
2,1101,648095,sub_out,2
3,1101,648096,block,1
4,1101,648096,foul_pers,1
5,1101,648096,made3_jump,1
6,1101,648096,miss3_jump,2
7,1101,648096,reb_def,1
8,1101,648096,sub_in,3
9,1101,648096,sub_out,3


In [5]:
# path = './data/'
# player_ext = 'Players_2018.csv'
# event_ext = 'Events_2018.csv'
# teams_ext = 'Teams.csv'

# events = pd.read_csv(path+event_ext)
# players = pd.read_csv(path+player_ext)
# teams = pd.read_csv(path+teams_ext)


In [7]:
# roster = events.groupby(['EventTeamID','EventPlayerID'])['EventType'].count().reset_index()
# players.head()


,PlayerID,Season,TeamID,PlayerName
0,648095,2018,1101,COBB_CHASE
1,648096,2018,1101,FARQHAR_HAYDEN
2,648097,2018,1101,FARQUHAR_HAYDEN
3,648098,2018,1101,FOSTER_TEVIN
4,648099,2018,1101,FRANKLIN_JAYLEN
